In [1]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
from scipy.stats import geom

# question 1

In [17]:
class Rejection_sampler:
    def __init__(self, p, I):  # Default value for m
        self.p = np.array(p)
        self.I = I
        self.N = len(p)
        
        if self.I <= 0 or self.I >= self.N :
            raise ValueError("I out of bound ( I does not belong to [1,...,N-1])")
        if np.count_nonzero(self.p) < self.I:
            raise ValueError("p is such that it's impossible to simulate a random variable following CB(p,I)")
        
        self.g = self.pdf_bernoulli()  # Bernoulli probability function
        self.f = self.pdf_cb()  # Conditional probability function
        self.M = self.compute_M()  # Compute M
        self._sample = None
        self._acceptances = None
    
    def compute_M(self):
        """Computes M based on given m and probability ratios"""
        res = 1
        for seq in self.generate_sequences(self.N, self.I):  # Corrected N and I
            pr = self.f(seq) / self.g(seq)  # Fix missing self.
            if pr > res:
                res = pr
        return res

    def pdf_cb(self, p = None, I = None):
        """Computes a conditional probability function"""
        if p == None:
            g = self.g
            p = self.p
        if I == None:
            I = self.I
        p = np.array(p)
        
        proba = 0
        for x in self.generate_sequences(self.N, I):
            proba += g(x)

        def f(x):
            return g(x) / proba if np.sum(x) == I else 0  # Use np.sum(x)

        return f

    @staticmethod
    def generate_sequences(N, I):
        """Generates all binary sequences of length N with I ones"""
        positions = itertools.combinations(range(N), I)
        sequences = []
        for pos in positions:
            seq = np.zeros(N, dtype=int)  # Start with all zeros
            seq[list(pos)] = 1  # Set the specified positions to 1
            sequences.append(seq)
        return np.array(sequences)
    
    def pdf_bernoulli(self, p=None):
        """Creates a Bernoulli probability function"""
        if p == None :
            p = self.p
        def g(x):
            return np.prod(np.where(x == 1, p, 1 - p))
        return g
    
    def sample(self,L=1):
        samples = []
        acceptances = []
        while len(samples) < L:
            attempt = 0
            while True:
                attempt += 1 
                X = np.array([np.random.binomial(1, p_i) for p_i in self.p])
                U = np.random.uniform(0,1)
                if U<=(self.f(X)/(self.g(X)*(self.M))):
                    samples.append(X)
                    acceptances.append(attempt)
                    break
        self._sample=np.array(samples)
        self._acceptances = np.array(acceptances)
    
    def plot_acceptance_density(self):
        """Plots the empirical density function and the theoretical Geometric(1/M) density."""
        if self._acceptances is None:
            raise Exception("No samples generated yet. Run sample() first.")

        unique, counts = np.unique(self._acceptances, return_counts=True)
        empirical_density = counts / np.sum(counts)  # Normalize to get probability values

        # Theoretical geometric distribution with success probability 1/M
        k_values = np.arange(1, max(unique) + 1)  # Range of values for plotting
        theoretical_density = geom.pmf(k_values, 1 / self.M)  # Geometric PMF

        plt.figure(figsize=(8, 5))
        
        # Empirical density as a bar plot
        plt.bar(unique, empirical_density, width=0.8, color='b', alpha=0.7, edgecolor='black', label="Empirical Density")
        
        # Theoretical geometric density as a line plot
        plt.plot(k_values, theoretical_density, 'r-o', markersize=4, label=f"Theoretical Geom(1/{self.M:.2f})", linewidth=1)

        plt.xlabel("Number of Attempts Before Acceptance")
        plt.ylabel("Probability Density")
        plt.title("Empirical vs Theoretical Density of Acceptance Attempts")
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.show()

### test exception

In [ ]:
sampler = Rejection_sampler([0]*10, 5)

ValueError: p is such that it's impossible to simulate a random variable following CB(p,I)

In [ ]:
sampler = Rejection_sampler([0.5]*10, -1)

ValueError: I out of bound ( I does not belong to [1,...,N-1])

# question 2

In [124]:
class exact_sampler:
    def __init__(self,p,I):
        self.p = np.array(p)
        self.I = I
        self.N = len(p)
        
        if self.I <= 0 or self.I >= self.N :
            raise ValueError("I out of bound ( I does not belong to [1,...,N-1])")
        if np.count_nonzero(self.p) < self.I:
            raise ValueError("p is such that it's impossible to simulate a random variable following CB(p,I)")
        
        self.q = self.compute_q()
        self._sample = None
    
    def compute_q(self):
        q = np.zeros((self.I+1,self.N))
        
        for n in range(self.N):
            q[0][n] = np.prod( 1 - self.p[n:])
        
        q[1][self.N - 1] = self.p[self.N - 1]
        
        for n in range(self.N - 2, -1, -1):
            for i in range(1, min(self.I, self.N - n) + 1):
                q[i][n] = self.p[n] * q[i - 1][n + 1] + (1 - self.p[n]) * q[i][ n + 1 ]
        
        return q
    
    def sample_one(self):
        X = np.zeros(self.N, dtype = int)
        i_n_min_1 = 0
        for n in range(self.N - 1):
            prob = self.p[n] * self.q[self.I - i_n_min_1 - 1][n+1] / self.q[self.I - i_n_min_1][n]
            if np.random.rand() <= prob:
                X[n] = 1
                i_n_min_1 += 1
                if i_n_min_1 == self.I:
                    break
        if i_n_min_1 != self.I:
            X[-1] = 1
        return X
    
    def sample(self, L = 1):
        res = []
        for _ in range(L):
            res.append(self.sample_one())
        self._sample = np.array(res)
    

In [125]:
sampler = exact_sampler([0.5] * 15, 4)

In [137]:
sampler.sample()
sampler._sample

array([[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1]])

# question 3

In [62]:
class MCMC_sampler:
    def __init__(self,p,I,):
        self.p = np.array(p)
        self.I = I
        self.N = len(p)
        
        if self.I <= 0 or self.I >= self.N :
            raise ValueError("I out of bound ( I does not belong to [1,...,N-1])")
        if np.count_nonzero(self.p) < self.I:
            raise ValueError("p is such that it's impossible to simulate a random variable following CB(p,I)")
        
        self.k = int(self.N*np.log(self.N))+1
        self._sample = None
        self._sequences = None
        
    
    def sample_one(self,k = None, conserve_sequence = False):
        res_seq = []
        if k is None:
            k=self.k
        X = np.zeros(self.N, dtype=int)
        X[:self.I ] = 1
        np.random.shuffle(X)
        w = self.p/(1-self.p)
        S1 = list(np.nonzero(X)[0])
        S0 = list(np.where(X == 0)[0])
        for m in range(k):
            np.random.shuffle(S0)
            np.random.shuffle(S1)
            i_0 = S0.pop()
            i_1 = S1.pop()
            alpha = min(1,w[i_0]/w[i_1])
            if np.random.rand() < alpha:
                if conserve_sequence : 
                    res_seq.append(X.copy())
                X[i_1],X[i_0] = 0,1
                
                S0.append(i_1)
                S1.append(i_0)
            else :
                S0.append(i_0)
                S1.append(i_1)
        
        if conserve_sequence:
            return (X, np.array(res_seq))
        return X
    
    def sample(self, L=1, k = None, conserve_sequence=False ):
        res = []
        res_seq = []
        for i in range(L):
            if conserve_sequence:
                X,seq = self.sample_one(k,True)
                res.append(X)
                res_seq.append(seq)
            else:
                X = self.sample_one(k)
                res.append(X)
        self._sample = np.array(res)
        if conserve_sequence:
            self._sequences = res_seq
        return

# question 4